# Level 1
# Client sends 2 of 3 parts of encrypted data to other parties via pickle

## Preparations

In [1]:
# modules
import socket  # creating server client connections for efficient communication
import pickle  # efficient file saving & loading
import time  # for stopping runtimes
import pandas as pd  # data mangling and transforming
import numpy as np  # matrix manipulations
from datetime import datetime  # for timestamps

In [2]:
# hostnames & ips
file = open('data/hosts&ips.txt', 'r')
hoips = eval(file.read())
file.close()

In [3]:
# read local encrypted data
file = open('data/encryptedData.pickle', 'rb')
d = pickle.load(file)

In [4]:
d.head()

,A,B,C
0,57768530708286,824345355159948,124827026505716
1,658587520123532,356636079786175,998658224838241
2,251549482772701,787047810069653,975284531905639
3,490525095894513,870105979653875,653250749199578
4,340071775594324,978282430575733,695527618577878


## Communication with other parties

### Establish connection to other parties (servers)

In [5]:
# start timing
start = time.time()
# general options
s0 = socket.socket()  # Create socket object 0
s1 = socket.socket()  # Create socket object 1
client = socket.gethostname()  # define current machine as client
port = 60387  # Reserve a port for your service.

In [6]:
# other parties as servers
servers = [key for key, value in hoips['hosts'].items() if value not in client]
server0 = hoips['hosts'][servers[0]]
server1 = hoips['hosts'][servers[1]]

In [7]:
# double-check parties
print('Server0 ('+servers[0]+'): '+server0)
print('Server1 ('+servers[1]+'): '+server1)
print('Client: '+client)

Server0 (chris): ip-172-31-38-9
Server1 (max): ip-172-31-44-44
Client: ip-172-31-46-89


In [8]:
# connect
s0.connect((server0, port))
s1.connect((server1, port))

### Send data

In [9]:
msg0 = pickle.dumps(d['A'])
msg1 = pickle.dumps(d['B'])

In [10]:
# send it
s0.send(msg0)
s1.send(msg1)

89288

In [11]:
# close sockets
s0.close()
s1.close()

### Save timer

In [12]:
# timer
end = time.time()
df_timer = pd.DataFrame(data={'phase': '1: Secret Sharing',
                              'timestamp': str(datetime.utcnow()),
                              'n_secrets': len(d),
                              'seconds': [round(end-start,2)]})

with open('data/timer.csv', 'a') as f:
    df_timer.to_csv(f, header=False)